In [ ]:
import ast

class PythonToJavaScript(ast.NodeVisitor):
    def __init__(self):
        self.js_code = ""
        self.indent = 0
        self.class_defs = []

    def indent_code(self, code):
        return "    " * self.indent + code

    def visit_Module(self, node):
        for stmt in node.body:
            self.visit(stmt)

    def visit_Expr(self, node):
        value = self.visit(node.value)
        if value:
            self.js_code += self.indent_code(value + "\n")

    def visit_Name(self, node):
        return node.id

    def visit_Constant(self, node):
        return repr(node.value)

    def visit_Assign(self, node):
        targets = ", ".join(self.visit(target) for target in node.targets if self.visit(target) is not None)
        value = self.visit(node.value)
        if targets and value:
            self.js_code += self.indent_code(f"let {targets} = {value};\n")

    def visit_BinOp(self, node):
        left = self.visit(node.left)
        right = self.visit(node.right)
        op = self.visit(node.op)
        return f"({left} {op} {right})"

    def visit_Add(self, node):
        return "+"

    def visit_Sub(self, node):
        return "-"

    def visit_Mult(self, node):
        return "*"

    def visit_Div(self, node):
        return "/"

    def visit_FunctionDef(self, node):
        args = ", ".join(arg.arg for arg in node.args.args if arg.arg != "self")
        self.js_code += self.indent_code(f"{node.name}({args}) {{\n")
        self.indent += 1
        for stmt in node.body:
            self.visit(stmt)
        self.indent -= 1
        self.js_code += self.indent_code("}\n")

    def visit_Return(self, node):
        value = self.visit(node.value)
        self.js_code += self.indent_code(f"return {value if value else ''};\n")

    def visit_If(self, node):
        self.js_code += self.indent_code(f"if ({self.visit(node.test)}) {{\n")
        self.indent += 1
        for stmt in node.body:
            self.visit(stmt)
        self.indent -= 1
        if node.orelse:
            self.js_code += self.indent_code("} else {\n")
            self.indent += 1
            for stmt in node.orelse:
                self.visit(stmt)
            self.indent -= 1
        self.js_code += self.indent_code("}\n")

    def visit_Compare(self, node):
        left = self.visit(node.left)
        ops = [self.visit(op) for op in node.ops]
        comparators = [self.visit(comp) for comp in node.comparators]
        return f" {ops[0]} ".join([left] + comparators)

    def visit_Gt(self, node):
        return ">"

    def visit_Eq(self, node):
        return "=="

    def visit_NotEq(self, node):
        return "!="

    def visit_For(self, node):
        iter_var = self.visit(node.target)
        iter_range = self.visit(node.iter)
        self.js_code += self.indent_code(f"for (let {iter_var} of {iter_range}) {{\n")
        self.indent += 1
        for stmt in node.body:
            self.visit(stmt)
        self.indent -= 1
        self.js_code += self.indent_code("}\n")

    def visit_While(self, node):
        self.js_code += self.indent_code(f"while ({self.visit(node.test)}) {{\n")
        self.indent += 1
        for stmt in node.body:
            self.visit(stmt)
        self.indent -= 1
        self.js_code += self.indent_code("}\n")

    def visit_Call(self, node):
        func = self.visit(node.func)
        args = ", ".join(filter(None, (self.visit(arg) for arg in node.args)))

        if func in self.class_defs:
            return f"new {func}({args})"
        if func == "print":
            return f"console.log({args});"
        return f"{func}({args})"

    def visit_Attribute(self, node):
        value = self.visit(node.value)
        if value == "self":
            value = "this"
        return f"{value}.{node.attr}"

    def visit_ClassDef(self, node):
        self.class_defs.append(node.name)
        self.js_code += self.indent_code(f"class {node.name} {{\n")
        self.indent += 1
        self.js_code += self.indent_code("constructor() {\n")
        self.indent += 1
        self.js_code += self.indent_code("this.historial = [];\n")
        self.indent -= 1
        self.js_code += self.indent_code("}\n")

        for stmt in node.body:
            if isinstance(stmt, ast.FunctionDef) and stmt.name != "__init__":
                method_name = stmt.name
                args = ", ".join(arg.arg for arg in stmt.args.args if arg.arg != "self")
                self.js_code += self.indent_code(f"{method_name}({args}) {{\n")
                self.indent += 1
                for substmt in stmt.body:
                    self.visit(substmt)
                self.indent -= 1
                self.js_code += self.indent_code("}\n")
        self.indent -= 1
        self.js_code += self.indent_code("}\n")

    def translate(self, python_code):
        tree = ast.parse(python_code)
        self.visit(tree)
        return self.js_code


In [ ]:
# Prueba del traductor
python_code = """
class Calculadora:

    def suma(self, a, b):
        resultado = a + b
        return resultado

    def resta(self, a, b):
        resultado = a - b
        return resultado

    def factorial(self, n):
        if n == 0:
            return 1
        else:
            resultado = n * self.factorial(n - 1)
            return resultado

# Uso de la calculadora
calc = Calculadora()
print(calc.suma(10, 5))
print(calc.resta(10, 3))
print(calc.factorial(5))

"""

translator = PythonToJavaScript()
js_code = translator.translate(python_code)
print(js_code)